In [1]:
import os
import time
import pandas as pd
from web3 import HTTPProvider, Web3
from pachira import *

In [2]:
chain = Net.POLYGON
platform = Platform.SUSHI
contract = JSONContract.UniswapV2Pair

json_rpc_url = os.environ.get(RPC.get_key(chain), RPC.get_rpc(chain))
web3 = Web3(HTTPProvider(json_rpc_url))
web3.middleware_onion.clear()

In [3]:
# Get contracts
abi_path = platform + '/' + contract + '.json'
Pair = ABILoad().get_contract(web3, abi_path)
swap_filt = Filter.create_filter(address=None, event_types=[Pair.events.Swap])  # Listen events from any smart contract

In [4]:
#ABILoading().get_abi_by_filename(abi_path)

In [5]:
reorg_mon = JSONRPCReorganizationMonitor(web3, check_depth=3)
reorg_mon.load_initial_block_headers(block_count=5)
processed_events = set()
latest_block = reorg_mon.get_last_block_live()
chain_reorg_resolution = reorg_mon.update_chain()
start, end = chain_reorg_resolution.get_read_range()
start = start - 25
print(f'latest_block {latest_block}, start {start}, end {end}')

latest_block 61383744, start 61383719, end 61383744


In [6]:
rEvents = ReadEvents().apply(web3, start_block=start, end_block=end, filter=swap_filt)

In [7]:
events = {}
evt: LogResult
for k, evt in enumerate(rEvents):
    # How to uniquely identify EVM logs
    key = evt["blockHash"] + evt["transactionHash"] + evt["logIndex"]
    topics = evt["topics"]
    arguments = Conversion().decode_data(evt["data"])

    event = {}
    event['chain'] = chain
    event['contract'] = contract.lower()
    event['type'] = evt["event"].event_name.lower()
    event['platform'] = platform
    event['address'] = evt["address"]
    event['tx_hash'] = evt["transactionHash"]
    event['blk_num'] = evt["blockNumber"]
    event['timestamp'] = evt["timestamp"]
    event['details'] = {}
    event['details']['web3_type'] = evt["event"]
    event['details']['token0'] = Conversion().convert_uint256_hex_string_to_address(topics[1])
    event['details']['token1'] = Conversion().convert_uint256_hex_string_to_address(topics[2])
    event['details']['amount0In'] = Conversion().convert_int256_bytes_to_int(arguments[0]) 
    event['details']['amount1Out'] = Conversion().convert_int256_bytes_to_int(arguments[3])  
    
    events[k] = event
    if key not in processed_events:
        print(f"Swap at block:{evt['blockNumber']:,} tx:{evt['transactionHash']}")
        processed_events.add(key)
else:
    print(".")


Swap at block:61,383,719 tx:0x8054d4b8a0ed84f99dc11cd4ec8b6d5b3761313d4fb08d4ae5c136e7f489d976
Swap at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Swap at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Swap at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Swap at block:61,383,722 tx:0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5
Swap at block:61,383,723 tx:0xbc9145c16919ba7f0eada3c717119aa25570d2a65461ea2361f1c93c1a9a6d1e
Swap at block:61,383,723 tx:0xe04b26d41f5195c58e22eeb7dd8bfa1b639f8fa9a3c391c91856de168bcb0770
Swap at block:61,383,727 tx:0xe07df91bda2aafd8fd002fd93427eea75b213ed0e7a9e36020722905f944a443
Swap at block:61,383,728 tx:0xe53f27e7da540e553ab574c94ccf7b4f5d21abb925cf78222153144df818a64e
Swap at block:61,383,729 tx:0xe07d830e39df882e8ec165ae928eba00769faaca9df513ac40e3a133bec519bf
Swap at block:61,383,729 tx:0x39378445f4c8239e3c59

In [8]:
df_events = pd.DataFrame.from_dict(events, orient='index') 
df_events

,chain,contract,type,platform,address,tx_hash,blk_num,timestamp,details
0,polygon,uniswapv2pair,swap,sushi,0x6a34bc58416381ad92aaae770256ed09ce2b2ec9,0x8054d4b8a0ed84f99dc11cd4ec8b6d5b3761313d4fb0...,61383719,1725384899,{'web3_type': <class 'web3._utils.datatypes.Sw...
1,polygon,uniswapv2pair,swap,sushi,0x0712323f8451cf7acc1141083baa60cc70dc32a8,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sw...
2,polygon,uniswapv2pair,swap,sushi,0xd6251d670b5fdf270bfea6de285003304dd5d177,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sw...
3,polygon,uniswapv2pair,swap,sushi,0x09b647132a26fb0813257656c165b03f8ee84e54,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sw...
4,polygon,uniswapv2pair,swap,sushi,0x3db7148e24be957a6229404c3f7a5fdc948ae543,0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b...,61383722,1725384905,{'web3_type': <class 'web3._utils.datatypes.Sw...
5,polygon,uniswapv2pair,swap,sushi,0x882df4b0fb50a229c3b4124eb18c759911485bfb,0xbc9145c16919ba7f0eada3c717119aa25570d2a65461...,61383723,1725384907,{'web3_type': <class 'web3._utils.datatypes.Sw...
6,polygon,uniswapv2pair,swap,sushi,0x70294d7aa244bd342c536f9b502152564057162e,0xe04b26d41f5195c58e22eeb7dd8bfa1b639f8fa9a3c3...,61383723,1725384907,{'web3_type': <class 'web3._utils.datatypes.Sw...
7,polygon,uniswapv2pair,swap,sushi,0x07a01030cc9b5379492485ba13b471a28d590d49,0xe07df91bda2aafd8fd002fd93427eea75b213ed0e7a9...,61383727,1725384915,{'web3_type': <class 'web3._utils.datatypes.Sw...
8,polygon,uniswapv2pair,swap,sushi,0x2e73e20c20ee938c94ae8928efe51244de7a7753,0xe53f27e7da540e553ab574c94ccf7b4f5d21abb925cf...,61383728,1725384919,{'web3_type': <class 'web3._utils.datatypes.Sw...
9,polygon,uniswapv2pair,swap,sushi,0xee321334ea2eacb78f04904ac4733cd301f53c6a,0xe07d830e39df882e8ec165ae928eba00769faaca9df5...,61383729,1725384921,{'web3_type': <class 'web3._utils.datatypes.Sw...


In [9]:
events

{0: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x6a34bc58416381ad92aaae770256ed09ce2b2ec9',
  'tx_hash': '0x8054d4b8a0ed84f99dc11cd4ec8b6d5b3761313d4fb08d4ae5c136e7f489d976',
  'blk_num': 61383719,
  'timestamp': 1725384899,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xedf6066a2b290C185783862C7F4776A2C8077AD1',
   'token1': '0x5fed7EA22dB17DC56eCF80761941c71563a572b6',
   'amount0In': 0,
   'amount1Out': 0}},
 1: {'chain': 'polygon',
  'contract': 'uniswapv2pair',
  'type': 'swap',
  'platform': 'sushi',
  'address': '0x0712323f8451cf7acc1141083baa60cc70dc32a8',
  'tx_hash': '0xbec00f74dee6b5a4ab89ec0144944a42d0eea3d25c2b233752fe954cc383d7c5',
  'blk_num': 61383722,
  'timestamp': 1725384905,
  'details': {'web3_type': web3._utils.datatypes.Swap,
   'token0': '0xd396e3b790F0fa79C80c69BD48127DB988A56bEb',
   'token1': '0xD6251d670B5FDf270BFeA6DE285003304DD5d177',
   'amount0In': 85242841327091,
   